<h1 style="text-align: center; font-size: 24pt">ICR : Mini-Projet</h1>
<h2 style="text-align: center; font-size: 18pt">Rapport</h2>
<h2 style="text-align: center; font-size: 18pt">Loïc Piccot - 21.05.2025</h2>

---

# 1. Introduction

## 1.1. Objectifs

Ce projet a pour objectif d'implémenter un **service de messagerie sécurisé** reposant sur la cryptographie. Cette messagerie offre la particularité d'envoyer des messages *"dans le futur"*, c'est à dire que ceux-ci ne pourront être lus qu'à partir d'une date donnée. Voici les principales caractéristiques du service :
- L'accès au service repose sur une architecture **client/serveur**.
- Les utilisateurs peuvent se connecter au serveur depuis n'importe quel appareil à l'aide d'un identifiant unique basé sur la paire **(Nom d'utilisateur/Mot de passe)**. Le mot de passe peut-être modifié.
- Les données sont **chiffrées de bout en bout**. Le serveur ne connait pas le contenu des messages et n'a pas la possibilité de les déchiffrer (modèle  *"honest but curious"*).
- Un utilisateur **peut télécharger ses messages, avant la date de déverrouillage**, mais ne pourra pas les déchiffrer tant que cette date n'est pas atteinte. Il peut cependant consulter à tout moment la date de disponibilité.
- Les messages sont **authentifiés et non-répudiables**.
- Le service est conçu pour supporter un nombre d'utilisateur très élevé.

Pour la réalisation pratique de ce projet, pour des questions de simplicité et de rapidité d'implémentation, c'est le language Python qui a été retenu. Et pour l'implémentation des fonctions cryptographique, nous utiliserons la librairie libsodium, plus précisément son binding python : [pyNaCl](https://pypi.org/project/PyNaCl/).

## 1.2. Architecture
L’architecture cryptographique du service repose sur une combinaison de primitives symétriques et asymétriques afin de garantir **confidentialité, intégrité et authenticité** des messages.

Premièrement, **avant tout échange**, le canal de communication client/serveur est sécurisé par le protocol TLS 1.3. Cela permet de garantir la confidentialité et l'intégrité de la communication entre les entitées.

Deuxièmement, lors de l’enregistrement d’un utilisateur, son mot de passe n’est jamais stocké en clair sur le serveur. Il est transformé en un **tag** vérificateur de mot de passe à l’aide d’une fonction de dérivation cryptographique (password hashing), renforcée par un sel aléatoire. Ce mécanisme empêche toute récupération directe du mot de passe, même en cas d’accès aux données du serveur.

Troisièmement, Lorsqu’un message est envoyé, une clé symétrique unique est générée aléatoirement. On utilise cette clé pour réaliser un chiffrement authentifié des données. Ce dernier garantie la confidentialité des données. Ce processus ajoute également une certaine forme d'authenticité puisque seul le détenteur de la clé peut chiffrer les données. Enfin, la génération d'un MAC prévient de toute modification des données cela garantie leur intégrité. Certaines métadonnées nécessaires au traitement du message sont laissées en clair sous forme de données authentifiées *(AAD – Additional Authenticated Data)*. Ces informations comprennent l’expéditeur, le destinataire, ainsi que la date de déverrouillage du message.

Enfin, pour garantir que seul le destinataire pourra lire le message, la clé symétrique est chiffrée à l’aide de sa clé publique via un algorithme de cryptographie asymétrique. Le serveur ne possède pas les clés privées des utilisateurs, ce qui l’empêche de déchiffrer la clé symétrique et par extension le contenu des messages. Cependant, la date de déchiffrement étant transmise en claire, le serveur peut décider de transmettre, où non, la clé de chiffrement au destinataire en fonction de la date de déverrouillage fixée par l'émetteur. 

# 2. Sécuriation du canal de communication

Même si le contenu des messages est chiffré de bout en bout, les metadonnées échangées en clair peuvent données des informations cruciales à un attaquant. Celle-ci contiennent, le nom d'utilisateur, le hash du mot de passe, le sel associé, la date et le destinataire des messages, ainsi que le contenu même des messages qui, bien que chiffré, pourrait-être stocké par un adversaire passif qui écouterait simplement le canal. C'est pourquoi il est déterminant pour la sécurité du service d'utiliser un protocol pour chiffrer les communications entre le client et le serveur.

![TLS](figures/TLS.png)

Dans ce but, c'est le protocol **TLS 1.3** qui a été retenu. Celui-ci permet :

- D'authentifier le serveur afin de se prémunir des attaques de type Man-in-the-middle
- De protéger l'intégriter du canal pour se prémunir d'éventuelles injections ou manipulations des requettes
- De rajouter une couche d'abstraction du materiel puisque le handshake permet de définir des algorithmes adaptés aux appareils utilisés.
- De se défendre éfficacement contre tout adversaire passif.

L’implémentation du protocole TLS n’étant pas requise dans le cadre de ce projet, et les solutions locales en Python n’étant pas pleinement adaptées au contexte de l'application, j’ai choisi de ne pas intégrer cette fonctionnalité à mon implémentation.

# 3. Enregistrement et Authentification utilisateur

Les utilisateurs doivent être capables de se connecter au serveur depuis n'importe quel appareil. Cela implique qu'aucune donné ne peut être stocké dans ces appareilles si ce n'est l'application elle même. C'est pourquoi, il est nécessaire de mettre en place des procédures d'enregistrement et d'authentification basés uniquement sur le mot de passe. Cela permettra à l'utilisateur de demander, dans un second temps, ses informations au serveur.

De plus, il est important que le serveur n'ai pas accès au mot de passe en clair, cela permet de réduire les risques en cas de fuite de données du serveur. Pour cela, à la place d'envoyer le mot de passe en clair au serveur, il est intéressant d'utiliser une fonction de dérivation pour extraire un tag de vérification du mot de passe. C'est donc ce tag ainsi que le sel utilisé pour le générer qui sont stockés sur le serveur.

La section 3.1 présente les choix cryptographique tandis que la section 3.2 présente le fonctionnement détaillé des protocols d'enregistrement, d'authentification et de changement de mot de passe d'un utilisateur auprès du serveur. Enfin la section 3.3 présente brievement l'implémentation retenue pour la partie cryptographique uniquement.

## 3.1. Choix cryptographiques

### 3.1.1. Argon2id

Pour le stockage de mot de passe, il est généralement recommandé d'utiliser une PBKDF (Password Base Key Derivation Function) dédiée à la sécurité des mots de passe, plutôt qu'une simple fonction de hachage comme SHA-256. Pour cette application, nous choisirons Argon2 pour les raisons suivantes :
- Les PBKDF sont plus lentes et couteuses en ressource ce qui est une bonne chose pour ralentir considérablement une attaque par *"brute-force"*
- Argon2 est propose de nombreux paramètres (voir section 3.1.2) pour adapter au besoin le ralentissement ou le cout en ressources
- Argon2 est recommandé, a une construction simple, une meilleure preuve de sécurité que ses concurents
- Argon2 est implémenté et même le choix par défaut dans la librairie libsodium choisit pour ce projet

    Plus précisément, nous choisirons la version `Argon2id` qui est un compromis (hybrid) entre :
- Argon2d: qui maximise la résistance contre les attaques GPU
- Argon2i: qui maximise la résistance aux "*side-channel attacks*"

### 3.1.2. Paramètres

- **Nombre de threads** : Il est recommandé d'en utiliser le maximum possible pour obliger l'attaquant à utiliser le même nombre, sous peine d'être serieusement ralenti (accélère aussi la connexion)
- **Taille mémoire** : La taille nécessaire à stocker les résultats intermédiaires d'Argon2. Ici le calcul est effectué en interne du device utilisateur et non sur le serveur. Le nombre de requettes de connexions simultanées n'est donc pas un facteur limitant. La taille mémoire dépend donc uniquement de la place qu'il est acceptable d'utiliser sur le device utilisateur.
- **Temps de connexion** : Il faut adapter le paramètre de complexité d'Argon2 pour s'approcher du temps de connexion souhaitable. Plus ce temps est élevé, plus la sécurité est renforcée (au détriment d'une expérience utilisateur légèrement dégradée). Etant donné qu'il s'agit d'une application d'échange de message potentiellement sensibles, on va forcer **2s** de temps de connexion.


### 3.1.3. Risques

Les risques liés à cette construction pour l'authentification du client sont :
- Si un attaquant connait le nom d'un utilisateur et son tag de vérification, il pourra alors s'identifier à la place de l'utilisateur.
- Dans le cas précédent, si le mot de passe n'est pas suffisament sûr, l'attaquant pourra faire une attaque par brut-force pour retrouver le mot de passe réel de l'utilisateur (potentiellement réutiliser dans d'autres services)
- Si un attaquant récupère uniquement le sel mais est capable de faire de multiples requettes au serveur il pourra aussi brut-forcer les mots de passe les plus faibles pour tenter une identification
- Si un attaquant récupère le nom et le token de session d'un utilisateur connecté au serveur, il pourra usurper l'identité de l'utilisateur pour la session active. 

## 3.2. Protocols
### 3.2.1. Enregistrement
![enregistrement](figures/enregistrement.png)
- L'utilisateur génère aléatoirement un sel
- L'utilisateur hache son mot de passe avec **Argon2** : $\tau = argon2(pwd, salt)$
- L'utilisateur envoie ses informations au serveur
- Le serveur vérifie la disponibilité du nom utilisateur
- Le serveur stocke les informations: $user_{\text{infos}} = (username, \tau, salt)$

### 3.2.2. Authentification
![login](figures/login.png)
- L'utilisateur envoie une requette au serveur avec son username
- Le serveur répond avec le sel associé
- L'utilisateur recalcul $\tau' = argon2(pwd, salt)$
- L'utilisateur transmets $(username, \tau')$ au serveur
- Le serveur vérifie et si le $\tau'$ reçu correspond bien au $\tau$ stocké lors de l'enregistrement.
- Le serveur valide l'authentification et génère un token de session pour l'utilisateur

### 3.2.3.  Changement de mot de passe
- L'utilisateur effectue la procédure d'identification
- L'utilisateur génère un nouveau sel aléatoire $salt'$
- L'utilisateur hache son nouveau mot de passe avec *Argon2* : $\tau' = argon2(pwd', salt')$
- L'utilisateur fournit ses données de session ainsi que son nouveau mot de passe $(username, token, pwd')$
- Le serveur vérifie le token de session
- Le serveur remplace les anciennes valeurs par celles reçues $(\tau, salt) \leftarrow (\tau', salt')$


## 3.3. Implémentation

Pour l'implémentation, nous utilisons donc le package `pwhash` de libsodium et sa fonction argon2id. Avec une taille de clé standard de 32 bytes soit 256 bits.
Les paramètres de coût mémoire `memlimit` et de coût d'opération `opslimit` sont fixés à SENSITIVE.

Pour la génération du sel, nous utilisons le package `utils.random` afin de générer 16 bytes soit 128 bits aléatoires.

```python
from nacl.pwhash import argon2id 
from nacl import secret, utils

def hash_password(password: bytes, salt: bytes) -> bytes:
    return argon2id.kdf(size=secret.SecretBox.KEY_SIZE,
                        password=password,
                        salt=salt,
                        opslimit=argon2id.OPSLIMIT_SENSITIVE,
                        memlimit=argon2id.MEMLIMIT_SENSITIVE)

def generate_salt() -> bytes: 
    return utils.random(argon2id.SALTBYTES)
```

Enfin, nous générons un token de session temporaire de 16 bytes généré à l'aide de la librairie python `secrets`. Celle-ci utilise le meilleur générateur de nombre aléatoire disponible sur l'appareil. Par exemple /dev/urandom sur linux.

```python
import secrets

def gen_token() -> str:
    return secrets.token_hex(16)
```


# 4. Chiffrement authentifié

## 4.1. Choix cryptographiques
## 4.2. Protocols
### 4.2.1. Envoie de message
- L'émetteur demande la clé publique du destinataire (voir 5.2.2)
- L'émetteur génère une clé symétrique
- L'émetteur prépare les données authentifiés contenant l'émetteur le destinataire et la date de déverrouillage du message $(sender | receiver | date_{unlock})$
- L'émetteur génère un nonce aléatoire associé au message
- L'émetteur utilise XChaCha20-Poly1305 pour chiffrer de manière authentifié : le contenu, les données authentifiés et le nonce. chiffre le contenu du message et génère un tag basé sur le nonce

## 4.3. Implémentation

# 5. Cryptographie asymétrique

## 5.1. Choix cryptographiques
## 5.2. Protocols

### 5.2.1. Génération de clés

### 5.2.2 Récupération de la clé publique du destinataire
- L'emetteur s'identifie sur le serveur et récupère son $token$ de session
- L'emetteur demande la clé publique du destinataire via $username$
- Le serveur vérifie la session et vérifie l'existence du destinataire
- Le serveur retourne la clé publique du destinataire